In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
import keras


from keras import backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("train.csv")

In [3]:
dtest = pd.read_csv("test.csv")

In [4]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data =  data.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [6]:
data.head()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,88.53,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,76.26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(4209, 369)

In [8]:
dtest

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dtest =  dtest.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [10]:
dtest.head() 

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
dtest.shape

(4209, 368)

In [12]:
X_train, y_train = data.values[0:3499,1:], data.values[0:3499, :1].ravel()
X_valid, y_valid = data.values[3500:4208,1:], data.values[3500:4208, :1].ravel()
X_test = dtest.values[:, :]

In [13]:
print("X_train----",X_train.shape)
print("y_train----",y_train.shape)      
print("X_valid----",X_valid.shape)
print("y_valid----",y_valid.shape)
print("X_test-----",X_test.shape)

X_train---- (3499, 368)
y_train---- (3499,)
X_valid---- (708, 368)
y_valid---- (708,)
X_test----- (4209, 368)


In [14]:
X_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [15]:
y_train[0]

130.81

In [16]:
# y_train = min_max_scaler.fit_transform(y_train)


In [17]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [19]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences

In [20]:
timesteps 

184

In [21]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_valid.reshape((X_valid.shape[0], subsequences, 1, timesteps, 1))
X_test  = X_test.reshape((X_test.shape[0], subsequences, 1, timesteps, 1))

In [22]:
X_train.shape

(3499, 2, 1, 184, 1)

In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=50)

In [27]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(64))
modelConvLSTM.add(Dense(32))
modelConvLSTM.add(Dense(1))
modelConvLSTM.compile(optimizer='adam', loss='mse')
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=700, verbose=2, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 3499 samples, validate on 708 samples
Epoch 1/700
 - 34s - loss: 10287.8766 - val_loss: 9645.3753
Epoch 2/700
 - 21s - loss: 9529.9907 - val_loss: 7993.5027
Epoch 3/700
 - 21s - loss: 6649.7814 - val_loss: 3300.6846
Epoch 4/700
 - 21s - loss: 1436.7710 - val_loss: 1031.3773
Epoch 5/700
 - 21s - loss: 1089.6176 - val_loss: 186.1676
Epoch 6/700
 - 21s - loss: 326.9648 - val_loss: 455.2619
Epoch 7/700
 - 21s - loss: 406.2818 - val_loss: 134.1756
Epoch 8/700
 - 21s - loss: 213.8799 - val_loss: 265.9983
Epoch 9/700
 - 21s - loss: 205.1158 - val_loss: 132.6763
Epoch 10/700
 - 21s - loss: 192.5088 - val_loss: 136.2464
Epoch 11/700
 - 21s - loss: 172.8127 - val_loss: 152.5013
Epoch 12/700
 - 21s - loss: 173.2925 - val_loss: 133.6840
Epoch 13/700
 - 21s - loss: 168.2749 - val_loss: 130.2755
Epoch 14/700
 - 21s - loss: 166.8993 - val_loss: 136.2322
Epoch 15/700
 - 21s - loss: 166.6879 - val_loss: 133.2881
Epoch 16/700
 - 21s - loss: 165.9325 - val_loss: 130.4821
Epoch 17/700
 - 21s - lo

Epoch 142/700
 - 21s - loss: 89.0139 - val_loss: 62.7601
Epoch 143/700
 - 21s - loss: 88.6318 - val_loss: 55.1429
Epoch 144/700
 - 21s - loss: 88.4791 - val_loss: 53.2129
Epoch 145/700
 - 21s - loss: 86.8328 - val_loss: 55.9953
Epoch 146/700
 - 21s - loss: 86.3850 - val_loss: 56.6816
Epoch 147/700
 - 21s - loss: 86.1541 - val_loss: 56.2934
Epoch 148/700
 - 21s - loss: 85.4269 - val_loss: 61.8919
Epoch 149/700
 - 21s - loss: 85.4902 - val_loss: 55.0792
Epoch 150/700
 - 21s - loss: 84.7595 - val_loss: 51.8488
Epoch 151/700
 - 21s - loss: 84.9142 - val_loss: 55.0425
Epoch 152/700
 - 21s - loss: 84.4977 - val_loss: 54.8991
Epoch 153/700
 - 21s - loss: 84.1954 - val_loss: 58.1087
Epoch 154/700
 - 21s - loss: 84.2042 - val_loss: 58.0961
Epoch 155/700
 - 21s - loss: 84.1108 - val_loss: 56.2299
Epoch 156/700
 - 21s - loss: 83.2827 - val_loss: 58.3119
Epoch 157/700
 - 21s - loss: 83.1873 - val_loss: 56.9416
Epoch 158/700
 - 21s - loss: 83.0122 - val_loss: 60.8720
Epoch 159/700
 - 21s - loss: 83

Epoch 286/700
 - 21s - loss: 76.6489 - val_loss: 48.3571
Epoch 287/700
 - 21s - loss: 76.8731 - val_loss: 47.8294
Epoch 288/700
 - 21s - loss: 76.9683 - val_loss: 47.3731
Epoch 289/700
 - 21s - loss: 77.4788 - val_loss: 48.5509
Epoch 290/700
 - 21s - loss: 78.0975 - val_loss: 57.7284
Epoch 291/700
 - 21s - loss: 77.4487 - val_loss: 55.6522
Epoch 292/700
 - 21s - loss: 76.8700 - val_loss: 54.0966
Epoch 293/700
 - 21s - loss: 76.3649 - val_loss: 53.5355
Epoch 294/700
 - 21s - loss: 76.2876 - val_loss: 51.1777
Epoch 295/700
 - 21s - loss: 76.1397 - val_loss: 52.1214
Epoch 296/700
 - 21s - loss: 76.4149 - val_loss: 48.3436
Epoch 297/700
 - 21s - loss: 76.3876 - val_loss: 51.6416
Epoch 298/700
 - 21s - loss: 76.5114 - val_loss: 54.0414
Epoch 299/700
 - 21s - loss: 76.1113 - val_loss: 53.4481
Epoch 300/700
 - 21s - loss: 76.5746 - val_loss: 51.1600
Epoch 301/700
 - 21s - loss: 76.0667 - val_loss: 49.2054
Epoch 302/700
 - 21s - loss: 76.0413 - val_loss: 47.4068
Epoch 303/700
 - 21s - loss: 76

Epoch 430/700
 - 21s - loss: 74.7271 - val_loss: 54.5111
Epoch 431/700
 - 21s - loss: 75.0675 - val_loss: 50.5861
Epoch 432/700
 - 21s - loss: 74.7335 - val_loss: 47.9814
Epoch 433/700
 - 21s - loss: 74.7854 - val_loss: 47.7696
Epoch 434/700
 - 21s - loss: 75.1552 - val_loss: 54.4907
Epoch 435/700
 - 21s - loss: 75.8167 - val_loss: 52.4561
Epoch 436/700
 - 21s - loss: 75.7754 - val_loss: 46.3332
Epoch 437/700
 - 21s - loss: 75.3220 - val_loss: 52.6952
Epoch 438/700
 - 21s - loss: 74.5566 - val_loss: 54.4781
Epoch 439/700
 - 21s - loss: 75.0435 - val_loss: 51.9074
Epoch 440/700
 - 21s - loss: 74.4568 - val_loss: 49.8369
Epoch 441/700
 - 21s - loss: 75.0604 - val_loss: 53.9276
Epoch 442/700
 - 21s - loss: 75.8793 - val_loss: 59.9342
Epoch 443/700
 - 21s - loss: 74.8279 - val_loss: 50.7102
Epoch 444/700
 - 21s - loss: 74.6342 - val_loss: 47.7391
Epoch 445/700
 - 21s - loss: 75.3950 - val_loss: 54.8177
Epoch 446/700
 - 21s - loss: 75.4450 - val_loss: 48.8494
Epoch 447/700
 - 21s - loss: 76

Epoch 574/700
 - 21s - loss: 74.6894 - val_loss: 52.8373
Epoch 575/700
 - 21s - loss: 73.9548 - val_loss: 46.7977
Epoch 576/700
 - 21s - loss: 74.5216 - val_loss: 50.0947
Epoch 577/700
 - 21s - loss: 73.9293 - val_loss: 55.4294
Epoch 578/700
 - 21s - loss: 74.2811 - val_loss: 51.3503
Epoch 579/700
 - 21s - loss: 73.7593 - val_loss: 47.8535
Epoch 580/700
 - 21s - loss: 73.9197 - val_loss: 51.9811
Epoch 581/700
 - 21s - loss: 73.9724 - val_loss: 58.8205
Epoch 582/700
 - 21s - loss: 75.1276 - val_loss: 54.2888
Epoch 583/700
 - 21s - loss: 74.2486 - val_loss: 48.7904
Epoch 584/700
 - 21s - loss: 74.4890 - val_loss: 47.3097
Epoch 585/700
 - 21s - loss: 74.4955 - val_loss: 64.9761
Epoch 586/700
 - 21s - loss: 75.3930 - val_loss: 49.9223
Epoch 587/700
 - 21s - loss: 73.7595 - val_loss: 47.5297
Epoch 588/700
 - 21s - loss: 75.9734 - val_loss: 46.9443
Epoch 589/700
 - 21s - loss: 75.5225 - val_loss: 64.9961
Epoch 590/700
 - 21s - loss: 75.9571 - val_loss: 50.1516
Epoch 591/700
 - 21s - loss: 74

In [28]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.6445103495323207,
 'Max Error': 39.666272277832036,
 'Mean Absolute Error': 6.176845682866156,
 'Mean Squared Error': 57.53231258193506,
 'Root Mean Squared Error': 7.585005773362012,
 'Median Absolute Error': 5.680847320556644,
 'R² Score': 0.5632014342417464,
 'Mean Absolute Percentage Error': 6.049541816471797}

In [29]:
ConvLSTMpred = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))


In [30]:
ConvLSTMpred 

array([ 77.78055,  93.80936,  77.5048 , ...,  93.32718, 111.14896,
        95.83975], dtype=float32)